## Evaluation Set Generation für RAG-System

In [0]:
%pip install -q mlflow mlflow[databricks] databricks-agents
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import mlflow
from databricks.agents.evals import generate_evals_df, estimate_synthetic_num_evals
from pyspark.sql.functions import (
  col, concat_ws, collect_list, count, first, max, length
)

In [0]:
# ====================================================================
# 1. Document-Level mit intelligenter Chunk-Zusammenführung
# ====================================================================
def chunk_merger(chunk_table):
  """
  Führt alle Chunks pro Dokument zu einem Text zusammen
  """
  merged_df = (
    chunk_table
      .groupBy('document_id')
      .agg(concat_ws(' ', collect_list('chunk')).alias('content'),
           first('url').alias('doc_uri'),
           count('chunk').alias('num_chunks'))
  ).select(
    col('content'), col('doc_uri'), col('num_chunks')
  )
  # Nur Dokumente mit genügend Content
  filtered_df = merged_df.filter((length(col('content')) > 500) & (col('num_chunks') > 1))
  return filtered_df

In [0]:
# ====================================================================
# 2. Agent Konfiguration
# ====================================================================
def agent_configuration():
    """
    Allgemeine Agent-Beschreibung
    """
    agent_description = """
    Der Agent ist ein RAG-Chatbot, der Fragen zu Produkten, Dienstleistungen oder anderen Themen beantwortet. Der Agent hat Zugriff auf eine umfangreiche Sammlung von Dokumenten, einschließlich:
    - Produktbeschreibungen
    - Preislisten und Tarife
    - Technische Spezifikationen
    - Verträge und Bedingungen
    - Weitere relevante Dokumentationen
    """
    
    question_guidelines = """
    # Nutzer-Personas
    - Kunden: Suchen schnelle, klare Antworten zu Produkten, Services oder anderen relevanten Themen
    - Vertriebsmitarbeiter: Benötigen detaillierte Produktinformationen, Preisinformationen und Kampagnendaten für Gespräche mit Kunden
    - Technische Ansprechpartner: Haben spezifische Fragen zu technischen Details, Anforderungen und Fehlerbehebungen
    
    # Beispielfragen
    - Was sind die Hauptmerkmale des Produkts?
    - Wie unterscheidet sich Produkt A von Produkt B?
    - Was sind die aktuellen Preise und Tarife?
    - Welche technischen Anforderungen muss man erfüllen, um dieses Produkt zu verwenden?
    - Was muss ich tun, wenn ich technische Probleme habe?

    # Zusätzliche Richtlinien
    - Fragen sollten klar, präzise und zum Thema passend formuliert sein
    - Fragen sollten auf natürliche Weise und mit einer klaren, freundlichen Sprache gestellt werden
    """
    
    return agent_description, question_guidelines

In [0]:
# ====================================================================
# 3. EVALUATION SET ERSTELLUNG
# ====================================================================
def evaluation_set(chunk_table, num_evals=20):
  """
  Erstellt einen Evaluation-Set
  """
  print("📑 Schritt 1: Führe Chunks zu Dokumenten zusammen...")
  merged_df = chunk_merger(chunk_table)
  print("📑 Schritt 2: Lade Agent-Konfiguration...")
  agent_description, question_guidelines = agent_configuration()
  print("📑 Schritt 2: Erstelle Evaluation-Set...")
  #num_evals = estimate_synthetic_num_evals(
  #  merged_df,
  #  eval_per_x_tokens = 800
  #)
  try:
    evals_df = generate_evals_df(
        merged_df,
        num_evals=num_evals,
        agent_description=agent_description,
        question_guidelines=question_guidelines
    )
    print(f"✅ Erfolgreich {evals_df.count()} Evaluations generiert!")  
  except Exception as e:
      print(f"❌ Fehler bei der Generierung: {str(e)}")
      return None
  return evals_df

In [ ]:
chunk_table = spark.sql("""
    SELECT DISTINCT chunk, url, document_id
    FROM chunks_test
    LIMIT 100
  """)
#display(chunk_table)

In [ ]:
evals = evaluation_set(chunk_table)
#display(evals)

In [0]:
results = mlflow.evaluate(
  model="endpoints:/databricks-meta-llama-3-3-70b-instruct",
  data=evals,
  model_type="databricks-agent",
)
#display(results.tables['eval_results'])